# NanoPyx-SRRF
A python reimplementation of the original [NanoJ-SRRF](https://github.com/HenriquesLab/NanoJ-SRRF) algorithm

In [1]:
# Load the dataset to analyse

import os

import ipywidgets as widgets
import skimage
import stackview
from IPython.display import display

from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

cwd = os.getcwd()
image_folder = "datasets"
image_files = []
EDM = ExampleDataManager()
example_datasets = EDM.list_datasets()

_path = os.path.join("..", image_folder)
if os.path.exists(_path):
    image_files += find_files(_path, ".tif")
if os.path.exists(image_folder):
    image_files += find_files(image_folder, ".tif")
image_files += ["Example dataset: "+dataset for dataset in example_datasets]

# Create a GUI
gui = EasyGui("srrf")

def on_button_load_data_clicked(b):
    global dataset_original
    if gui["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_original = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
    else:
        dataset_original = skimage.io.imread(gui["data_source"].value)

gui.add_label("Select data to use:")
gui.add_dropdown("data_source", options=image_files,
                 value="Example dataset: "+example_datasets[4])
gui.add_button("load_data", description="Load data")
gui["load_data"].on_click(on_button_load_data_clicked)
gui.show()

Label(value='Select data to use:', layout=Layout(width='50%'), style=LabelStyle(description_width='initial'))

Dropdown(index=4, layout=Layout(width='50%'), options=('Example dataset: EMBL_TubulinGFP', 'Example dataset: I…

Button(description='Load data', layout=Layout(width='50%'), style=ButtonStyle())

In [2]:
# Crop the dataset

nFrames = dataset_original.shape[0]
h = dataset_original.shape[1]
w = dataset_original.shape[2]

dataset = dataset_original

def on_button_crop_clicked(b):
    f_start = gui["f_start"].value
    f_stop = gui["f_stop"].value
    x_start = gui["x_start"].value
    x_stop = gui["x_stop"].value
    y_start = gui["y_start"].value
    y_stop = gui["y_stop"].value
    assert f_start < f_stop
    assert x_start < x_stop
    assert y_start < y_stop

    global dataset
    dataset = dataset_original[
        f_start:f_stop+1,
        y_start:y_stop+1,
        x_start:x_stop+1
    ]

gui = EasyGui("srrf")
gui.add_int_slider("x_start", description="X start:", min=0, max=w, value=0)
gui.add_int_slider("x_stop", description="X stop:", min=0, max=w, value=w-1)
gui.add_int_slider("y_start", description="Y start:", min=0, max=h, value=0)
gui.add_int_slider("y_stop", description="Y stop:", min=0, max=h, value=h-1)
gui.add_int_slider("f_start", description="Frame start:", min=0, max=nFrames-1, value=0)
gui.add_int_slider("f_stop", description="Frame stop:", min=0, max=nFrames-1, value=nFrames-1)
gui.add_button("crop", description="Crop")
gui["crop"].on_click(on_button_crop_clicked)
gui.show()



IntSlider(value=0, description='Frame start:', layout=Layout(width='50%'), max=499, style=SliderStyle(descript…

IntSlider(value=499, description='Frame stop:', layout=Layout(width='50%'), max=499, style=SliderStyle(descrip…

IntSlider(value=0, description='X start:', layout=Layout(width='50%'), max=128, style=SliderStyle(description_…

IntSlider(value=127, description='X stop:', layout=Layout(width='50%'), max=128, style=SliderStyle(description…

IntSlider(value=0, description='Y start:', layout=Layout(width='50%'), max=128, style=SliderStyle(description_…

IntSlider(value=127, description='Y stop:', layout=Layout(width='50%'), max=128, style=SliderStyle(description…

Button(description='Crop', layout=Layout(width='50%'), style=ButtonStyle())

In [3]:
stackview.slice(dataset, continuous_update=True, zoom_factor=2, zoom_spline_order=0)

In [4]:

gui = EasyGui("srrf")
gui.add_float_slider("ring_radius", description="Ring Radius:", min=0.1, max=3.0, value=0.5, remember_value=True)
gui.add_int_slider("magnification", description="Magnification:", min=1, max=10, value=4)
gui.add_checkbox("do_drift_correction", description="Do drift correction", value=False)
gui.add_label("-=-= Time-Lapse =-=-")
gui.add_int_text("frames_per_timepoint", description="Frames per time-point (0 - auto)", value=0)
gui.show()



FloatSlider(value=0.5, description='Ring Radius:', layout=Layout(width='50%'), max=3.0, min=0.1, style=SliderS…

IntSlider(value=4, description='Magnification:', layout=Layout(width='50%'), max=10, min=1, style=SliderStyle(…

Checkbox(value=False, description='Do drift correction', layout=Layout(width='50%'), style=CheckboxStyle(descr…

Label(value='-=-= Time-Lapse =-=-', layout=Layout(width='50%'), style=LabelStyle(description_width='initial'))

IntText(value=0, description='Frames per time-point (0 - auto)', layout=Layout(width='50%'), style=Description…